Live view data prepare
---

# load photos

In [1]:
import PIL
import os
import numpy as np

live_view_folder = '../photos/live_view/'
photo_files = os.listdir(live_view_folder)

def import_image(path):
    image_data = False
    try:
        with PIL.Image.open(path) as an_image:
            image_data = an_image.copy()
        return image_data
    except:
        pass

live_images = [
    import_image( os.path.join(live_view_folder, file))
    for file in photo_files if file.endswith('.jpg')         
]

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../photos/live_view/'

In [ ]:
import datetime as dt
import pandas as pd

In [ ]:
#the last few don't exist yet.
live_images = live_images[:911]

In [ ]:
all_live_images = np.array([
    np.array(image)
    for image in live_images
])

In [ ]:
#get some statistics on each image
sums = [
    np.sum(image)
    for image in live_images
]

import matplotlib.pyplot as plt

plt.plot(sums)
plt.title('total brightness

In [ ]:
np.unique([np.array(image).shape for image in live_images])


all images are 1024x720x3

In [ ]:
exif_data = [
    [thing
    for thing in live_image.getexif().items()]
    for live_image in live_images
]

exif_data[:4]

In [ ]:
#image dataframe.

codes = [296, 34665, 271, 272, 306, 531, 282, 283]

col_to_code = dict(zip(
    range(8),
    codes
))

descriptions = [
    'Resolution unit',
    'ExifIFDPointer',
    'Make',
    'Model',
    'date',
    'YCbCrPositioning',
    'XResolution',
    'YResolution'    
]

code_to_description = dict(zip( codes, descriptions))


col_to_description = dict(zip(range(8), descriptions))

idf = pd.DataFrame(exif_data)

idf.rename(columns = col_to_description, inplace=True)

idf = idf.map(lambda entry: entry[1])
idf

In [ ]:
idf['date'] = pd.to_datetime(idf['date'], format = '%Y:%m:%d %H:%M:%S')

idf

In [ ]:
idf.info()

In [ ]:
idf.plot('date', 'Resolution unit', kind = 'scatter', figsize = (12,5))

In [ ]:
idf

In [ ]:
np.array(live_images[0]).shape

In [ ]:
#save image shape
for index in idf.index:
    # print(index)
    idf.loc[index, 'image_shape'] = str(np.array(live_images[int(index)]).shape)

#save some statistics about each image

#get the sum of the numpy array data
for index in idf.index:
    idf.loc[index, 'sum_brightness'] = np.array(live_images[index]).sum()
    
idf

In [ ]:
idf.plot(x='date', y='sum_brightness')

In [ ]:
idf.loc[idf['sum_brightness'] == idf['sum_brightness'].drop([349, 350]).min()]

In [ ]:
def display_value_counts(dataframe, column, vc_limit = 5, normalize = True):
    value_counts = dataframe[column].value_counts(normalize = normalize)
    if len(value_counts) <= vc_limit:
        print(value_counts)
    else:
        print(f"{column} has \t{len(value_counts)} distinct values")

hbar = '='*50

print(hbar)
for column in idf.columns:
    display_value_counts(idf, column)
    print(hbar)


In [ ]:
idf['date'].value_counts(ascending = False)[:20]

In [ ]:
sum_brightness_vcs = idf['sum_brightness'].value_counts(ascending = False)
#for every sum_brightness vcs greater than 1, get the entries with that value.

for thing in sum_brightness_vcs[sum_brightness_vcs:
    pass

In [ ]:
#strange. there are six duplicate sets of times.
#but only 3 duplicate sets of sum brightness. So these aren't all just copies.

for this_sum in sum_brightness_vcs.index:
    if sum_brightness_vcs[this_sum] > 1:
        print(idf.loc[ idf['sum_brightness'] == this_sum ])

In [ ]:
idf.loc[this_date_mask].index

In [ ]:
#ok, so the three sets with the same sum are all taken at the same time. These may be repeat images.

#so the other three sets are photos taken at different times with the same sum brightness. What're the odds of that??

date_vcs = idf['date'].value_counts(ascending = False)

same_date_index = []

for this_date in date_vcs.index:
    if date_vcs[this_date] > 1:
        this_date_mask = idf['date'] == this_date
        print(idf.loc[ this_date_mask ])
        same_date_index = same_date_index +  list(idf.loc[this_date_mask].index)
same_date_index

In [243]:
#show all same-date guys
idf.loc[same_date_index]

#add a new column linking them to the other
for index, record in idf.iterrows():
    this_date = record['date']
    this_brightness = record['sum_brightness']
    shared_date_index = idf.loc[(idf.drop(index)['date'] == this_date)].index
    shared_brightness_index = idf.loc[(idf.drop(index)['sum_brightness'] == this_brightness)].index
    idf.loc[index, 'shares_date_with'] = str(shared_date_index)
    idf.loc[index, 'shares_brightness_with'] = str(shared_brightness_index)

idf

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).